In [3]:
%run contactsScraper.py


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=829039132881-tm658s48p30jafdjcsdgtlm08h6b06ak.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&access_type=offline&response_type=code

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver

Authentication successful.
Storing credentials to C:\Users\Anthony\scripts\Contacts-Scraper\.credentials\sheets.googleapis.Scraper-X.json
Done
Done
KEYS
Done
Done
RECORDS COLLECTED
DATAFRAMES READY


In [8]:
import pandas as pd
import scraperModelGS as smgs

In [10]:
orgRecords

[{'Directory link': 'https://www.hfma.org/contact/',
  'Last Call': '',
  'Link 2': '',
  'Link 3': '',
  'Link 4': '',
  'Organization': 'Healthcare Financial Management Association',
  'Request Time': ''},
 {'Directory link': 'https://www.nraringoffreedom.com/meet-the-staff',
  'Last Call': '',
  'Link 2': '',
  'Link 3': '',
  'Link 4': '',
  'Organization': 'The National Rifle Association (NRA)',
  'Request Time': ''},
 {'Directory link': 'https://www.aap.org/en-us/Pages/Chapter-Executive-Director-Contact-List-and-Websites.aspx',
  'Last Call': '',
  'Link 2': '',
  'Link 3': '',
  'Link 4': '',
  'Organization': 'American Academy of Pediatrics (AAP)',
  'Request Time': ''},
 {'Directory link': 'http://www.usga.org/about/our-leadership.html',
  'Last Call': '',
  'Link 2': '',
  'Link 3': '',
  'Link 4': '',
  'Organization': 'United States Golf Association',
  'Request Time': ''},
 {'Directory link': 'http://www.api.org/about/leadership-team',
  'Last Call': '',
  'Link 2': '',
  

In [12]:
from selenium import webdriver
import pandas as pd
import scraperModelGS as smgs
from bs4 import BeautifulSoup

path_to_chromedriver = '/Users/Anthony/scripts/Contacts-Scraper/Drivers/chromedriver' # change path as needed


In [5]:
def findOrgRecord(organization):
    for org in orgRecords:
        if organization == org['Organization']:
            return org

In [8]:
orgRecords.index(findOrgRecord('American Academy of Pediatrics (AAP)'))

2

In [61]:
class DirectoryManager(object):
    ## The goal of this class is to manage the directory in the enviroment not to be one!
    ## the directory as well as it access and packaging functions will opporate as utility functions, these
    ## the routines of this class will call those utility functions
    def __init__(self, orgRecords):
        self.orgRecords = orgRecords
        #self.browser = webdriver.Chrome(path_to_chromedriver)
        
    def findOrgRecord(self, organization):
        for org in self.orgRecords:
            if organization == org['Organization']:
                return org
            
    def get_organizations(self):
        return [x['Organization'] for x in self.orgRecords]
            
    def orgRecordIndex(self, orgRecord):
        return self.orgRecords.index(orgRecord)
    
    def linkList(self, orgRecord):
        lis = [orgRecord['Directory link'], orgRecord['Link 2'], orgRecord['Link 3'], orgRecord['Link 4']]
        return lis[:lis.index('')]
    
    def writeRecordRow(self, row, index):
        """Google Sheets API Code.
        """
        credentials = get_credentials()
        http = credentials.authorize(smgs.httplib2.Http())
        discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                        'version=v4')
        service = smgs.discovery.build('sheets', 'v4', http=http,
                                  discoveryServiceUrl=discoveryUrl)

        spreadsheet_id = '1p1LNyQhNhDBNEOkYQPV9xcNRe60WDlmnuiPp78hxkIs'
        value_input_option = 'RAW'
        rangeName = 'Org Leadership Websites!F' + str(index + 2)
        values = row
        body = {
              'values': values
        }

        result = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=rangeName,
                                                        valueInputOption=value_input_option, body=body).execute()

        return result
    

        

In [62]:
class OrgSession(DirectoryManager):
    def __init__(self, orgRecords, browser):
        DirectoryManager.__init__(self, orgRecords)
        self.browser = browser
        
    def processSession(organization):
        self.organization = organization
        self.orgRecord = DirectoryManager.findOrgRecord(self.organization)
        self.sessionIndex = DirectoryManger.orgRecordIndex(self.orgRecord)
       
        ## Get all links for Session Instance
        self.links = DirectoryManager.linkList(self, self.orgRecord)
        
        ## Retreve Queries
        self.orgQueries = [OrgQuery(link, self.browser) for link in links]
            
        ## Collect and write collection record to sheet
        callTime = self.orgQueries[-1].get_callTimeStr()
        totalTime = sum([query.get_responseTime() for query in self.orgQueries])
        
        res = DirectoryManager.writeRecordRow([[callTime, totalTime]], self.sessionIndex)
        
        ## Return Query Objects
        return self.orgQueries

In [63]:
from bs4 import BeautifulSoup
import time
import datetime as dt

class OrgQuery(object):
    def __init__(self, link, browser):
        self.link = link
        start = time.clock()
        self.query = browser.get(link)
        self.pageSource = self.query.page_source
        self.soup = BeautifulSoup(self.pageSource, 'lxml')
        self.responseTime = time.clock() - start
        self.callTime = dt.datetime.now()
        
    def get_query(self):
        return self.query
    
    def get_pageSource(self):
        return self.pageSource
    
    def get_soup(self):
        return self.soup
    
    def get_responseTime(self):
        return self.responseTime
    
    def get_callTime(self):
        return self.callTime
        
    def get_callTimeStr(self):
        return self.callTime.strftime('%a %b %d, %Y  %H:%M:%S')
        

In [56]:
a = [1,2,4]

In [57]:
a[-1]

4

In [58]:
sum(a)

7

In [59]:
sum([2,3,4])

9